In [96]:
import os
import pandas as pd
import pickle

from dotenv import load_dotenv
from routingGA import RoutingGA
from GALib import GALib
from data.addresses import addresses
from distancesAPI import MapsAPI

%reload_ext autoreload
%autoreload 2

load_dotenv(override=True)

True

In [97]:
refetch_distances = False
calculate_time = False

In [98]:
mapsAPI = MapsAPI(os.getenv('API_KEY'))
address_chunks = MapsAPI.split_in_chunks(addresses, 25)
distances = mapsAPI.split_distance_request(address_chunks, calculateTime = calculate_time) if refetch_distances else MapsAPI.get_from_cache()
distances = MapsAPI.convert_to_symmetric(distances)

In [99]:
df = pd.DataFrame(distances)

In [100]:
with open('cache/distance_matrix', 'rb') as f:
    best_ever = pickle.load(f)

In [127]:
routingGA = RoutingGA(popSize=200, qtyLocations=len(distances) - 1, qtyRoutes=3,
                          maxGenerations=800, selectionK=5, mutationRate=0.05, optRate=0.02,distances=distances)
                           
lib = GALib(routingGA=routingGA,
            libPath=os.getenv('LIB_PATH'))

In [159]:
result = lib.run()
print(routingGA.calculate_total_distance(result))
print([len(x) for x in result])
result

7424
[24, 9, 2]


[[0,
  25,
  26,
  4,
  16,
  2,
  28,
  23,
  19,
  24,
  18,
  10,
  1,
  3,
  13,
  11,
  9,
  21,
  17,
  27,
  22,
  31,
  20,
  32],
 [0, 12, 15, 7, 30, 8, 6, 5, 29],
 [0, 14]]

In [104]:
best_n2 = [[0, 15, 26, 9, 23, 19, 28, 31, 21, 2, 25, 32, 22, 17, 27, 18, 10, 14],
 [0, 11, 6, 8, 5, 7, 16, 12, 4, 20, 3, 30, 29, 24, 13, 1]]

In [147]:
best_n3 = [[0, 29, 31, 19, 28, 15, 23, 30, 24, 22, 18],
 [0, 3, 16, 21, 14, 13, 10, 27, 25, 6, 7, 9, 17, 8, 20, 2, 11, 1, 12],
 [0, 5, 26, 4, 32]]

In [110]:
best_n4 = [[0, 30, 29, 31],
 [0, 11, 15, 12, 14, 5, 22, 1, 13],
 [0, 4, 3, 28, 2, 20, 19, 17, 27, 25, 24, 10, 8, 7, 9, 6, 32, 26, 16, 21, 18, 23]]

In [88]:
routingGA.calculate_total_distance(result)

10982

In [89]:
# with open('cache/best_ever', 'wb'):
#     pickle.dumps(best)

In [90]:
# route_addresses = [[addresses[y] for y in x] for x in best]
# results = mapsAPI.split_directions_request(route_addresses[0])
# mapsAPI.save_to_image(results, f"route_test")

In [161]:
route_addresses = [[addresses[y] for y in x] for x in best_n4]
imageCount = 1
for route in route_addresses:
    directions = mapsAPI.directions(route[0], route[0], route[1: 25])
    mapsAPI.save_to_image(directions, f"n4_route_{imageCount}")
    imageCount += 1

In [ ]:
plot = mapsAPI.plot_markers(addresses)
with open('test.jpg', 'wb') as f:
    for chunk in plot:
        f.write(chunk)